In [2]:
import sys
sys.path.append('./venv/lib/python3.6/site-packages/')

import yaml
import weibopy as wb


with open('./credentials.yml') as f:
    cred = yaml.load(f)

oauth = wb.OAuthHandler(**cred['weibo']).authorize()
api = wb.API(oauth=oauth)
data, status = api.statuses.user_timeline()

Please load the link https://api.weibo.com/oauth2/authorize?response_type=code&client_id=3677147061&redirect_uri=https%3A%2F%2Fwww.greenpeace.org%2F&state=C4kIeUdY1QpE5FNsQWpfX6szLmfdwB to your browser and authorize weibo API access.
Enter the full callback URL: https://www.greenpeace.org/usa/?state=C4kIeUdY1QpE5FNsQWpfX6szLmfdwB&code=2aade97302f1abed30bee8af12d0cad0
UID: 1292378751
Access Token expires at: 2023/07/07 22:28:31


In [3]:
from utils import MongoWriter
writer = MongoWriter(**cred['mongo'])

In [4]:
from weibo_data import *

writer.get_since_id(collection='statuses')

4257970281382180

In [5]:
for s_data in data['statuses']:
    status = Status(s_data, write_obj=True, writer=writer)
    status.write(writer=writer) 

DuplicateKeyError: E11000 duplicate key error collection: weibo_data.statuses index: _id_ dup key: { : 4250426327276250 }

In [ ]:
from weibo_data import *
from utils import MongoWriter


def main(cred):
    writer = MongoWriter(**cred['mongo'])
    oauth = wb.OAuthHandler(**cred['weibo']).authorize()
    api = wb.API(oauth=oauth)
    data, _ = api.statuses.user_timeline()
    for s_data in data['statuses']:
        status = Status(s_data)
        print(status)
        status.write(writer=writer)


if __name__ == '__main__':
    with open('./credentials.yml') as f:
        cred = yaml.load(f)
    main(cred=cred)

In [ ]:
api = wb.API(oauth)
data, status = api.statuses.user_timeline(count=100)

In [ ]:
statuses = data.get('statuses', [])

In [ ]:
for status in statuses:
    print(status.get('created_at'), status.get('id'))

In [ ]:
data, status = api.statuses.user_timeline(count=100, max_id=4248766729509158, trim_user=1)

In [ ]:
statuses2 = data.get('statuses', [])

In [ ]:
for status in statuses2:
    print(status.get('created_at'), status.get('id'))

In [ ]:
status = statuses2[2]
status

In [ ]:
status.keys()

In [ ]:
data, _ = api.statuses.show(id='4243617021947926')

In [ ]:
data.keys()

In [ ]:
data['user']

In [ ]:
data, _ = api.users.show(uid='1292378751')

In [ ]:
data

In [ ]:
help(api.users.show)

In [ ]:
'screen_name', 'name', 'province', 'city', 'location', 'description', 'url', 'profile_image_url', 
'cover_image', 'cover_image_phone', 'profile_url', 'domain', 'gender', 'followers_count', 'friends_count', 
'pagefriends_count', 'statuses_count', 'video_status_count', 'favourites_count', 'created_at', 'geo_enabled',
'verified', 'insecurity', 'ptype', 'allow_all_comment', 'avatar_large', 'avatar_hd', 'verified_reason', 
'verified_state', 'verified_level', 'verified_type_ext', 'has_service_tel', 'verified_contact_name', 
'verified_contact_email', 'verified_contact_mobile', 'follow_me', 'like', 'like_me', 'online_status', 'bi_followers_count', 'lang', 'star', 'mbtype', 'mbrank', 'block_word', 'block_app', 'credit_score', 'user_ability', 'urank', 'story_read_state', 'vclub_member'

In [ ]:
'screen_name', 'name', 'province', 'city', 'location', 'description', 'url', 'profile_image_url',
'cover_image', 'cover_image_phone', 'profile_url', 'domain', 'gender', 'followers_count', 'friends_count',
'pagefriends_count', 'statuses_count', 'video_status_count', 'favourites_count', 'created_at', 'geo_enabled', 
'verified', 'insecurity', 'ptype', 'allow_all_comment', 'avatar_large', 'avatar_hd', 'verified_reason', 
'verified_state', 'verified_level', 'verified_type_ext', 'has_service_tel','verified_contact_name',
'verified_contact_email', 'verified_contact_mobile', 'follow_me', 'like', \
            'like_me', 'online_status', 'bi_followers_count', 'lang', 'star', 'block_word', 'block_app', 'credit_score',\
            'user_ability', 'urank', 'story_read_state',

In [ ]:
data, _ = api.comments.show(id='4250426327276250')

In [ ]:
comments = data['comments']
len(comments)

In [ ]:
for c in comments:
    print(c['created_at'], c['id'])

In [ ]:
data2, _ = api.comments.show(id='4250426327276250', max_id='4250427878887380')

In [ ]:
comments = data2['comments']

for c in comments:
    print(c['created_at'], c['id'])